In [5]:
import os
import cv2
import numpy as np
import torch
from torchvision import transforms
from torchvision.models.segmentation import deeplabv3_resnet101
from torchvision.transforms import Compose, ToTensor, Normalize
from PIL import Image
import matplotlib.pyplot as plt
from skimage import measure  # 연결된 컴포넌트 분석을 위한 모듈

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# [STEP 0] 이미지 로드
img_path = os.path.join(os.getenv('USERPROFILE'), 'Desktop', 'aiffel', 'segmetation', 'images', 'image3.png')
img_orig = cv2.imread(img_path)
img_rgb = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)

# [STEP 1] DeepLabv3로 인물 마스크 생성 후 가장 큰 인물만 선택
def largest_person_mask(image_bgr):
    model = deeplabv3_resnet101(pretrained=True).to(device).eval()
    transform = Compose([
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = transform(cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)).unsqueeze(0).to(device)
    
    with torch.no_grad():
        output = model(input_tensor)['out'][0]
        mask = output.argmax(0).byte().cpu().numpy()
    
    # 인물 클래스(15)에 해당하는 마스크 추출
    person_mask = (mask == 15).astype(np.uint8)
    
    # 연결된 컴포넌트 레이블링으로 서로 다른 인물 구분
    labels = measure.label(person_mask)
    regions = measure.regionprops(labels)
    
    if not regions:  # 인물이 없는 경우
        return np.zeros_like(person_mask)
    
    # 가장 큰 영역(인물) 선택
    largest_region = max(regions, key=lambda r: r.area)
    largest_person_mask = np.zeros_like(person_mask)
    largest_person_mask[labels == largest_region.label] = 1
    
    # 마스크 후처리로 잡음 제거 및 경계 부드럽게
    kernel = np.ones((5, 5), np.uint8)
    largest_person_mask = cv2.morphologyEx(largest_person_mask.astype(np.uint8), cv2.MORPH_CLOSE, kernel) * 255
    
    return largest_person_mask

# 마스크 생성 및 시각화 함수
def visualize_masks(core_mask, feather_mask, bg_mask, original_img):
    plt.figure(figsize=(15, 10))
    
    # 원본 이미지
    plt.subplot(2, 2, 1)
    plt.imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
    plt.title("Original Image")
    plt.axis('off')
    
    # Core 영역 시각화
    plt.subplot(2, 2, 2)
    plt.imshow(core_mask, cmap='gray')
    plt.title("Core Mask (Main Person)")
    plt.axis('off')

    # Feather 영역 시각화
    plt.subplot(2, 2, 3)
    plt.imshow(feather_mask, cmap='gray')
    plt.title("Feather Mask")
    plt.axis('off')

    # Background 영역 시각화
    plt.subplot(2, 2, 4)
    plt.imshow(bg_mask, cmap='gray')
    plt.title("Background Mask")
    plt.axis('off')

    plt.tight_layout()
    plt.show()

# 가장 큰 인물 마스크 생성 및 처리
# 차이점 - 마스크 생성 방식 (임계값 조정)
mask = largest_person_mask(img_orig)
mask_blur = cv2.GaussianBlur(mask, (31, 31), 0)# 블러 커널 크기 증가

# Core, Feather, Background 영역 분리
# Core/Feather 영역 분리 방식 개선
core_mask = (mask_blur > 150).astype(np.uint8) * 255  # 임계값 낮춤으로 더 많은 영역 포함
feather_mask = ((mask_blur > 5) & (mask_blur <= 150)).astype(np.uint8) * 255
bg_mask = (mask_blur <= 5).astype(np.uint8) * 255

# 마스크 시각화 호출
visualize_masks(core_mask, feather_mask, bg_mask, img_orig)

# [STEP 2] MiDaS로 depth map 생성 및 개선
# 차이점 -MiDaS 깊이 추정 + 양방향 필터링 추가
def estimate_depth_midas(img):
    midas = torch.hub.load("intel-isl/MiDaS", "DPT_Hybrid").to(device).eval()
    transform = torch.hub.load("intel-isl/MiDaS", "transforms").dpt_transform
    img_transformed = transform(img).to(device)
    
    with torch.no_grad():
        prediction = midas(img_transformed)
        depth = prediction.squeeze().cpu().numpy()
        # 깊이 맵 품질 개선을 위한 양방향 필터링
        depth_filtered = cv2.bilateralFilter(depth.astype(np.float32), d=9, sigmaColor=75, sigmaSpace=75)
        depth_normalized = cv2.normalize(depth_filtered, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        return depth_normalized

depth_map = estimate_depth_midas(img_rgb)

# 깊이 맵 시각화
plt.figure(figsize=(8, 8))
plt.imshow(depth_map, cmap='plasma')
plt.title("Depth Map")
plt.axis('off')
plt.colorbar(label='Depth')
plt.show()

# [STEP 3] 블러 및 블렌딩 적용
# 차이점 - 블렌딩 알고리즘 (선명도 보정)
def blur_with_feathering(img, depth_map, core_mask, feather_mask, bg_mask):
    depth_resized = cv2.resize(depth_map, (img.shape[1], img.shape[0]))
    
    # 강한 배경 블러 적용 (61x61 커널)
    img_blur = cv2.GaussianBlur(img, (61, 61), 0)  # 강한 블러 효과
    
    # 3채널 마스크 변환
    core_3ch = cv2.cvtColor(core_mask, cv2.COLOR_GRAY2BGR)
    feather_3ch = cv2.cvtColor(feather_mask, cv2.COLOR_GRAY2BGR)
    bg_3ch = cv2.cvtColor(bg_mask, cv2.COLOR_GRAY2BGR)
    
    # Feather 영역 블렌딩 - 선명도 증가
    feather_alpha = feather_mask.astype(np.float32) / 255.0
    feather_alpha = feather_alpha[:, :, np.newaxis] * 0.8 + 0.2  # 선명도 증가
    feather_blended = (img * feather_alpha + img_blur * (1 - feather_alpha)).astype(np.uint8)
    
    # 최종 합성
    final_img = np.zeros_like(img)
    final_img = np.where(core_3ch > 0, img, final_img)  # Core는 원본 유지
    final_img = np.where(feather_3ch > 0, feather_blended, final_img)  # Feather는 블렌딩
    final_img = np.where(bg_3ch > 0, img_blur, final_img)  # Background는 강한 블러
    
    return final_img
final_img = blur_with_feathering(img_orig, depth_map, core_mask, feather_mask, bg_mask)

# [STEP 4] 기본 배경 교체
def replace_background(img, new_bg_path, core_mask, feather_mask, bg_mask):
    # 새 배경 이미지 로드
    new_bg = cv2.imread(new_bg_path)
    
    # 원본 이미지 크기에 맞게 배경 조정
    new_bg = cv2.resize(new_bg, (img.shape[1], img.shape[0]))
    
    # 3채널 마스크 변환
    core_3ch = cv2.cvtColor(core_mask, cv2.COLOR_GRAY2BGR)
    feather_3ch = cv2.cvtColor(feather_mask, cv2.COLOR_GRAY2BGR)
    
    # Feather 영역 블렌딩을 위한 알파값 계산
    feather_alpha = feather_mask.astype(np.float32) / 255.0
    feather_alpha = feather_alpha[:, :, np.newaxis]
    
    # Feather 영역 블렌딩 - 원본과 새 배경 사이 자연스러운 전환
    feather_blended = (img * feather_alpha + new_bg * (1 - feather_alpha)).astype(np.uint8)
    
    # 최종 합성
    final_img = np.zeros_like(img)
    final_img = np.where(core_3ch > 0, img, final_img)  # Core는 원본 유지
    final_img = np.where(feather_3ch > 0, feather_blended, final_img)  # Feather는 블렌딩
    final_img = np.where((core_3ch == 0) & (feather_3ch == 0), new_bg, final_img)  # 나머지는 새 배경
    
    return final_img

# [STEP 5] 배경 교체2
def background_replace(img, new_bg_path, core_mask, feather_mask, depth_map):
    # 새 배경 이미지 로드
    new_bg = cv2.imread(new_bg_path)
    new_bg = cv2.resize(new_bg, (img.shape[1], img.shape[0]))
    
    # 원본 이미지와 새 배경의 색상 분포 조정 (색상 일관성)
    foreground_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    background_hsv = cv2.cvtColor(new_bg, cv2.COLOR_BGR2HSV)
    
    # 전경의 평균 색상 계산
    fg_mask = (core_mask > 0).astype(np.uint8)
    fg_hsv_mean = np.array([
        np.mean(foreground_hsv[:, :, 0][fg_mask > 0]),
        np.mean(foreground_hsv[:, :, 1][fg_mask > 0]),
        np.mean(foreground_hsv[:, :, 2][fg_mask > 0])
    ])
    
    # 배경의 평균 색상 계산
    bg_hsv_mean = np.array([
        np.mean(background_hsv[:, :, 0]),
        np.mean(background_hsv[:, :, 1]),
        np.mean(background_hsv[:, :, 2])
    ])
    
    # 색상 조정 가중치 (HSV 조정)
    color_adjustment = (fg_hsv_mean - bg_hsv_mean) * 0.3
    
    # 새 배경 색상 조정
    background_hsv[:, :, 0] = np.clip(background_hsv[:, :, 0] + color_adjustment[0], 0, 179)
    background_hsv[:, :, 1] = np.clip(background_hsv[:, :, 1] + color_adjustment[1], 0, 255)
    background_hsv[:, :, 2] = np.clip(background_hsv[:, :, 2] + color_adjustment[2], 0, 255)
    
    # HSV에서 BGR로 변환
    adjusted_bg = cv2.cvtColor(background_hsv, cv2.COLOR_HSV2BGR)
    
    # 깊이 맵 기반 그림자 효과 추가
    depth_resized = cv2.resize(depth_map, (img.shape[1], img.shape[0]))
    depth_norm = depth_resized / 255.0  # 정규화
    
    # 그림자 강도 (낮은 깊이 = 강한 그림자)
    shadow_strength = 0.7 * (1 - depth_norm)[:, :, np.newaxis]
    shadow_strength = shadow_strength * 0.4  # 그림자 강도 조절
    
    # 3채널 마스크
    core_3ch = cv2.cvtColor(core_mask, cv2.COLOR_GRAY2BGR) / 255.0
    feather_3ch = cv2.cvtColor(feather_mask, cv2.COLOR_GRAY2BGR) / 255.0
    
    # Feather 영역 블렌딩
    feather_alpha = feather_3ch
    
    # 그림자 적용 배경
    shadowed_bg = adjusted_bg * (1 - shadow_strength)
    
    # 최종 블렌딩
    blended = img * core_3ch + \
              img * feather_alpha + shadowed_bg * (1 - feather_alpha - core_3ch)
    
    return blended.astype(np.uint8)


# 새 배경 이미지 경로 설정
new_background_path = os.path.join(os.getenv('USERPROFILE'), 'Desktop', 'aiffel', 'segmetation', 'images', 'bg_image3.png')
# 만약 beach.jpg 파일이 없다면, 사용 가능한 다른 배경 이미지 경로로 변경하세요

replaced_bg_img = replace_background(
    img_orig, 
    new_background_path, 
    core_mask, 
    feather_mask, 
    bg_mask
)

# 고급 배경 교체 적용
background_replaced_img = background_replace(
    img_orig, 
    new_background_path, 
    core_mask, 
    feather_mask, 
    depth_map
)

# 원본과 결과 비교 시각화
plt.figure(figsize=(20, 8))

plt.subplot(1, 4, 1)
plt.imshow(cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB))
plt.title("Original Image")
plt.axis('off')

plt.subplot(1, 4, 2)
plt.imshow(cv2.cvtColor(final_img, cv2.COLOR_BGR2RGB))
plt.title("Feather Blending + Blur")
plt.axis('off')

plt.subplot(1, 4, 3)
plt.imshow(cv2.cvtColor(replaced_bg_img, cv2.COLOR_BGR2RGB))
plt.title("background Replace1")
plt.axis('off')

plt.subplot(1, 4, 4)
plt.imshow(cv2.cvtColor(background_replaced_img, cv2.COLOR_BGR2RGB))
plt.title("background Replace2")
plt.axis('off')

plt.tight_layout()
plt.show()



error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


**결과물1**
변경 전 이미지
![0.png](./0.png)
함수적용
![01.png](./01.png)
![02.png](./02.png)
적용 후 이미지
![03.png](./03.png)
'''

**결과물2**
함수적용
![001.png](./001.png)
![002.png](./002.png)
적용 후 이미지
![003.png](./003.png)

**결과물3**
함수적용
![1.png](./1.png)
![2.png](./2.png)
적용 후 이미지
![3.png](./3.png)

# 코드 진행 방식
[STEP 0] 이미지 로드
     ↓
[STEP 1] 인물 마스크 추출
     ↓
[STEP 2] 배경 복원 (피사체 제거 & inpaint)
     ↓
[STEP 3] 배경 스타일링 (채도, 하늘, 톤)
     ↓
[STEP 4] 피사체 부드럽게 처리
     ↓
[STEP 5] 재합성 (톤 맞추기, 그림자, 블렌딩)
     ↓
[OPTION] LUT / 전체 색감 조정
     ↓
출력: 배경 중심의 고급스러운 풍경 인물 사진

**[STEP 0] 이미지 로드**
cv2.imread로 이미지를 로드하고 RGB 형식으로 변환합니다.

이미지 경로는 사용자 환경변수(USERPROFILE)를 기반으로 설정됩니다.

**[STEP 1] 가장 큰 인물 마스크 생성**
DeepLabv3 모델(deeplabv3_resnet101)을 사용하여 이미지의 세그멘테이션 결과를 얻습니다.

인물 클래스(15)에 해당하는 마스크를 추출하고, skimage.measure 모듈로 연결된 컴포넌트 분석을 수행해 가장 큰 영역(인물)을 선택합니다.

선택된 마스크는 후처리를 통해 잡음을 제거하고 경계를 부드럽게 만듭니다.

**[STEP 2] 깊이 맵 생성**
MiDaS 모델(DPT_Hybrid)을 사용하여 이미지의 깊이 정보를 추출합니다.

양방향 필터링(cv2.bilateralFilter)을 통해 깊이 맵의 품질을 개선하고, 정규화하여 시각적으로 표현 가능한 형태로 변환합니다.

**[STEP 3] 블러 및 블렌딩 효과 적용**
Gaussian Blur를 사용해 배경에 강한 블러 효과를 적용합니다.

Core 영역(인물)은 원본 이미지를 유지하고, Feather 영역(경계)은 원본과 블러 이미지를 자연스럽게 블렌딩하며, 배경은 강한 블러만 적용됩니다.

**[STEP 4] 기본 배경 교체**
새 배경 이미지를 로드하고 원본 이미지 크기에 맞게 조정합니다.

Feather 영역은 원본과 새 배경 사이를 부드럽게 전환하며, Core 영역은 원본 이미지를 그대로 유지합니다.

**[STEP 5] 고급 배경 교체**
새 배경의 색상을 원본 이미지와 일치시키기 위해 HSV 색상 조정을 수행합니다.

깊이 맵 정보를 활용해 그림자 효과를 추가하여 더 현실감 있는 결과를 만듭니다.

최종적으로 Core, Feather, 그림자 효과가 반영된 새 배경 이미지를 합성합니다.

# 차이점:
**정확한 인물 선택**

연결된 컴포넌트 분석(skimage.measure) 추가로 다중 인물 환경에서 가장 큰 인물 정확히 선택

모폴로지 연산(cv2.MORPH_CLOSE)으로 마스크 경계 개선

**깊이 처리 개선**

MiDaS 깊이 맵 + 양방향 필터링 적용으로 노이즈 감소

깊이 기반 그림자 효과 추가

**자연스러운 합성**

HSV 색상 공간에서의 색상 보정

3채널 마스크와 계층적 블렌딩

Feather 영역 가중치 조정(0.8 → 0.2)으로 선명도 향상

**시각화 강화**

Core/Feather/Background 마스크 분리 표시

4-way 비교 시각화(원본, 블러, 기본 교체, 고급 교체)

**성능 최적화**

GPU 가속 지원 강화

큰 커널 사이즈(61x61) 사용으로 배경 블러 품질 향상